In [ ]:
# ===============================================
# Tâche 1 - Collecte & Préparation Multi-Source
# ANIP Challenge : Bénin
# ===============================================

import pandas as pd
from pandas_datareader import wb
import requests
from bs4 import BeautifulSoup
import os

# Création des groupes s'ils n'existent pas
os.makedirs("datasets/raw", exist_ok=True)
os.makedirs("datasets/final", exist_ok=True)

# ================================
# 1. Indicateurs Banque Mondiale
# ================================
start_year = 2000
end_year = 2023

indicators = {
    # Démographiques
    "SP.POP.TOTL": "population",
    "SP.URB.TOTL": "population_urbaine",
    "SP.POP.GROW": "taux_croissance_pop",
    
    # Économiques
    "NY.GDP.MKTP.CD": "gdp_usd",
    "FP.CPI.TOTL": "inflation",
    "SL.UEM.TOTL.ZS": "taux_chomage",
    "NE.EXP.GNFS.CD": "exportations",
    
    # Sociaux
    "SP.DYN.LE00.IN": "esperance_vie",
    "SE.ADT.LITR.ZS": "alphabetisation",
    "SE.PRM.ENRR": "scolarisation_primaire"
}

# Téléchargement de tous les indicateurs
df = wb.download(indicator=list(indicators.keys()), country="BJ", start=start_year, end=end_year)
df.rename(columns=indicators, inplace=True)

# Réinitialisation de l'index et calcul du PIB par habitant 
df.reset_index(inplace=True)
df["gdp_per_capita"] = df["gdp_usd"] / df["population"]

# Sauvegarde des données brutes
df.to_csv("datasets/raw/benin_multi_raw.csv", index=False)
print("Données brutes multi-source sauvegardées : datasets/raw/benin_multi_raw.csv")

# ================================
# 2. Scraping indicateurs sportifs
# ================================
url = "https://fr.wikipedia.org/wiki/B%C3%A9nin_aux_Jeux_olympiques"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Extraction du tableau des participations
tables = soup.find_all("table", {"class": "wikitable"})
sport_df = pd.DataFrame()

if tables:
    table = tables[0]  # Premier tableau
    sport_df = pd.read_html(str(table))[0]
    sport_df = sport_df.rename(columns={sport_df.columns[0]: "year"})
    sport_df["year"] = pd.to_numeric(sport_df["year"], errors="coerce")
    # Garder uniquement les années 2000-2023
    sport_df = sport_df[(sport_df["year"] >= start_year) & (sport_df["year"] <= end_year)]
    sport_df = sport_df.sort_values("year").reset_index(drop=True)
    # Exemple colonne : nombre de participants
    if "Participants" in sport_df.columns:
        sport_df.rename(columns={"Participants": "nb_participants_olympiques"}, inplace=True)
else:
    print("Aucun tableau sportif trouvé sur la page Wikipédia.")

# Merge des données sportives
if not sport_df.empty:
    df = df.merge(sport_df[["year", "nb_participants_olympiques"]], on="year", how="left")
else:
    df["nb_participants_olympiques"] = None

# ================================
# 3. Nettoyage et vérification
# ================================
df = df.sort_values("year").reset_index(drop=True)
df.dropna(subset=["population", "gdp_usd"], inplace=True)

# Sauvegarde finale
df.to_csv("datasets/final/benin_multi.csv", index=False)
print("Dataset final multi-source sauvegardé : datasets/final/benin_multi.csv")

# ================================
# 4. Glossaire des variables
# ================================
glossaire = pd.DataFrame({
    "Variable": [
        "year", "population", "population_urbaine", "taux_croissance_pop", 
        "gdp_usd", "gdp_per_capita", "inflation", "taux_chomage", "exportations",
        "esperance_vie", "alphabetisation", "scolarisation_primaire", "nb_participants_olympiques"
    ],
    "Nom complet": [
        "Année", "Population totale", "Population urbaine", "Taux de croissance de la population",
        "PIB", "PIB par habitant", "Inflation", "Taux de chômage", "Exportations",
        "Espérance de vie", "Taux d'alphabétisation adulte", "Taux de scolarisation primaire", "Participants aux JO"
    ],
    "Définition": [
        "Année de la mesure",
        "Nombre total d’habitants",
        "Nombre d’habitants en milieu urbain",
        "Variation annuelle de la population totale",
        "Produit Intérieur Brut en USD courants",
        "PIB par habitant = PIB / population",
        "Variation annuelle des prix à la consommation",
        "Proportion de la population active au chômage",
        "Valeur totale des exportations de biens et services",
        "Espérance de vie à la naissance",
        "Pourcentage d’adultes alphabétisés",
        "Taux de scolarisation dans l’enseignement primaire",
        "Nombre de participants béninois aux Jeux Olympiques"
    ],
    "Unité": [
        "année", "personnes", "personnes", "%", "USD", "USD/personne", "%", "%", "USD",
        "années", "%", "%", "personnes"
    ],
    "Source": [
        "Banque Mondiale"]*12 + ["Wikipedia"],
    "Période": ["2000-2023"]*13,
    "Géographie": ["Bénin"]*13
})

glossaire.to_csv("datasets/final/glossaire_variables.csv", index=False)
print("Glossaire sauvegardé : datasets/final/glossaire_variables.csv")

# ================================
# 5. Fichier anomalies
# ================================
anomalies = df[(df["population"] <= 0) | (df["gdp_usd"] <= 0) | (df["gdp_per_capita"] <= 0)]
anomalies.to_csv("datasets/final/anomalies.csv", index=False)
print("Fichier anomalies sauvegardé : datasets/final/anomalies.csv")


# ----------------------------
# Tâche 1 (suite) : enrichir benin_multi.csv
# ----------------------------

import pandas as pd
from pandas_datareader import wb
import os

os.makedirs("datasets/raw", exist_ok=True)
os.makedirs("datasets/final", exist_ok=True)

# 1) Chargement du dataset existant
base_fp = "datasets/final/benin_multi.csv"
df_base = pd.read_csv(base_fp)

# 2) Téléchargement des nouveaux indicateurs World Bank (2000-2023)
start_year = 2000
end_year = 2023
wb_indicators = {
    "SE.PRM.ENRR": "primary_enrolment_gross",   # School enrollment, primary (% gross)
    "SE.ADT.LITR.ZS": "adult_literacy_pct",    # Adult literacy rate (%)
    "SP.DYN.LE00.IN": "life_expectancy",       # Life expectancy at birth (years)
    "SH.DYN.MORT": "under5_mortality_per1000"  # Under-5 mortality rate (per 1000)
}

print("Téléchargement des indicateurs WDI...")
df_wb = wb.download(indicator=list(wb_indicators.keys()), country="BJ", start=start_year, end=end_year)

# Renommer les colonnes & réindexer
df_wb = df_wb.rename(columns=wb_indicators).reset_index()  

if 'date' in df_wb.columns:
    df_wb = df_wb.rename(columns={'date': 'year'})
if 'Year' in df_wb.columns:
    df_wb = df_wb.rename(columns={'Year': 'year'})

# Assurer que year est entier
df_wb['year'] = pd.to_numeric(df_wb['year'], errors='coerce').astype(int)

# Sauvegarde brute
df_wb.to_csv("datasets/raw/benin_wb_additional_raw.csv", index=False)
print("WDI brutes sauvegardées : datasets/raw/benin_wb_additional_raw.csv")

# 3) Fusionner avec le dataset de base
# Normaliser noms de colonnes
if 'country' in df_wb.columns:
    df_wb = df_wb.drop(columns=['country'])
# Merge sur 'year' (left join en prenant années déjà présentes dans df_base)
df_enriched = pd.merge(df_base, df_wb, on='year', how='left')

# 4) Indicateurs dérivés utiles
# Croissance annuelle population & PIB (en %)
df_enriched = df_enriched.sort_values("year").reset_index(drop=True)
df_enriched['pop_growth_pct'] = df_enriched['population'].pct_change() * 100
df_enriched['gdp_growth_pct'] = df_enriched['gdp_usd'].pct_change() * 100

# (re)calculer gdp_per_capita par sécurité
df_enriched['gdp_per_capita'] = df_enriched['gdp_usd'] / df_enriched['population']

# 5) Nettoyage simple
for col in ['primary_enrolment_gross', 'adult_literacy_pct', 'pop_growth_pct', 'gdp_growth_pct']:
    if col in df_enriched.columns:
        df_enriched[col] = pd.to_numeric(df_enriched[col], errors='coerce').round(2)

# 6) Sauvegarder dataset enrichi
out_fp = "datasets/final/benin_multi_enriched.csv"
df_enriched.to_csv(out_fp, index=False)
print(f"Dataset enrichi sauvegardé : {out_fp}")

# 7) Détecter anomalies simples (valeurs manquantes & bornes incohérentes)
missing_summary = df_enriched.isnull().sum()
anomalies_rows = df_enriched[
    (df_enriched['population'] <= 0) |
    (df_enriched['gdp_usd'] <= 0) |
    (df_enriched['gdp_per_capita'] <= 0) |
    (df_enriched['under5_mortality_per1000'] < 0) |
    (df_enriched['adult_literacy_pct'] > 150)
].copy()

# Sauvegarde résumé & anomalies
missing_summary.to_csv("datasets/final/missing_summary_2.csv", header=True)
anomalies_rows.to_csv("datasets/final/anomalies_2.csv", index=False)
print("missing_summary_2.csv et anomalies_2.csv générés dans datasets/final/")

# 8) Aperçu final
print("\nAperçu dataset enrichi (head):")
display(df_enriched.head())

print("\nRésumé des valeurs manquantes (colonnes principales) :")
print(missing_summary.loc[['population','gdp_usd','primary_enrolment_gross','adult_literacy_pct','life_expectancy','under5_mortality_per1000']])

# Ajoute lignes au glossaire existant (ou crée si introuvable)
glossaire_fp = "datasets/final/glossaire_variables.csv"
new_rows = [
    {"Variable":"primary_enrolment_gross", "Nom complet":"Scolarisation primaire (taux brut)", "Définition":"Gross enrolment ratio - primaire", "Unité":"%", "Source":"World Bank (WDI)", "Période":"2000-2023", "Géographie":"Bénin"},
    {"Variable":"adult_literacy_pct", "Nom complet":"Taux d'alphabétisation adulte", "Définition":"% d'adultes (15+) sachant lire et écrire", "Unité":"%", "Source":"World Bank (WDI)", "Période":"2000-2023", "Géographie":"Bénin"},
    {"Variable":"life_expectancy", "Nom complet":"Espérance de vie à la naissance", "Définition":"Espérance de vie totale (années)", "Unité":"années", "Source":"World Bank (WDI)", "Période":"2000-2023", "Géographie":"Bénin"},
    {"Variable":"under5_mortality_per1000", "Nom complet":"Mortalité <5 ans", "Définition":"Taux de mortalité des moins de 5 ans (par 1000 naissances)", "Unité":"pour 1000", "Source":"World Bank (WDI)", "Période":"2000-2023", "Géographie":"Bénin"}
]
new_df = pd.DataFrame(new_rows)

if os.path.exists(glossaire_fp):
    glossaire = pd.read_csv(glossaire_fp)
    glossaire = pd.concat([glossaire, new_df], ignore_index=True)
else:
    glossaire = new_df

glossaire.to_csv(glossaire_fp, index=False)
print("glossaire mis à jour :", glossaire_fp)




/tmp/ipykernel_8432/3641577614.py:41: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = wb.download(indicator=list(indicators.keys()), country="BJ", start=start_year, end=end_year)


Données brutes multi-source sauvegardées : datasets/raw/benin_multi_raw.csv
Aucun tableau sportif trouvé sur la page Wikipédia.
Dataset final multi-source sauvegardé : datasets/final/benin_multi.csv
Glossaire sauvegardé : datasets/final/glossaire_variables.csv
Fichier anomalies sauvegardé : datasets/final/anomalies.csv
Téléchargement des indicateurs WDI...
WDI brutes sauvegardées : datasets/raw/benin_wb_additional_raw.csv
Dataset enrichi sauvegardé : datasets/final/benin_multi_enriched.csv
missing_summary_tache2.csv et anomalies_tache2.csv générés dans datasets/final/

Aperçu dataset enrichi (head):


/tmp/ipykernel_8432/3641577614.py:169: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_wb = wb.download(indicator=list(wb_indicators.keys()), country="BJ", start=start_year, end=end_year)


,country,year,population,population_urbaine,taux_croissance_pop,gdp_usd,inflation,taux_chomage,exportations,esperance_vie,alphabetisation,scolarisation_primaire,gdp_per_capita,nb_participants_olympiques,primary_enrolment_gross,adult_literacy_pct,life_expectancy,under5_mortality_per1000,pop_growth_pct,gdp_growth_pct
0,Benin,2000,7221619,2768263,3.076653,3.519991e+09,74.331948,0.864,7.253049e+08,56.594,NaN,80.111214,487.424141,NaN,80.11,NaN,56.594,136.1,NaN,NaN
1,Benin,2001,7445596,2877723,3.054355,3.666223e+09,77.293552,0.765,7.284263e+08,56.654,NaN,87.407501,492.401500,NaN,87.41,NaN,56.654,132.8,3.10,4.15
2,Benin,2002,7675426,2996179,3.040108,4.194343e+09,79.217514,0.690,8.274017e+08,56.851,35.0,92.610580,546.463830,NaN,92.61,35.0,56.851,129.6,3.09,14.41
3,Benin,2003,7913070,3128511,3.049203,5.349258e+09,80.395671,0.747,9.973316e+08,57.092,NaN,96.328888,676.002878,NaN,96.33,NaN,57.092,126.5,3.10,27.54
4,Benin,2004,8159094,3266820,3.061731,6.190270e+09,81.098241,0.790,1.146039e+09,57.346,NaN,100.354599,758.695804,NaN,100.35,NaN,57.346,123.5,3.11,15.72



Résumé des valeurs manquantes (colonnes principales) :
population                   0
gdp_usd                      0
primary_enrolment_gross      2
adult_literacy_pct          18
life_expectancy              0
under5_mortality_per1000     0
dtype: int64
glossaire mis à jour : datasets/final/glossaire_variables.csv
